In [0]:
"""Importamos el drive para poder ejecutar los scripts"""
#Estamos trabajando todo de momento mediante drive
#Scripts, ficheros y documentos.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Cambiamos el directorio de trabajo a los ficheros de datos
cd '/content/drive/My Drive/Proyecto 2 SENSORES/Ficheros de datos/'

/content/drive/My Drive/Proyecto 2 SENSORES/Ficheros de datos


In [0]:
#Instalamos unas librerias necesarias
!pip install pyproj

     |████████████████████████████████| 10.4MB 2.0MB/s 


In [0]:
from pyproj import Proj, transform
from pyproj import Transformer
import datetime as dt
from datetime import datetime
import numpy as np
import json,csv
import os
import pandas as pd 
import matplotlib.pyplot as plt
import math
import ast

In [0]:
"""
se le pasa el archivo y devuelve la columna con el tiempo formateado
"""
def timeFormated(date):
  dt_string = datetime.strptime(str(date),"%Y-%m-%dT%H:%M:%S.%f+01:00") 
  return dt_string.strftime("%Y-%m-%d %H:%M:%S")

In [0]:
"""
se le pasa el archivo, lee las coordenas y devuelve las coordendas transformadas
"""
def coordsColumnsTransformation(data):
  transformer = Transformer.from_crs('epsg:25830','epsg:4326',always_xy=True)
  puntos = data.coordinates
  lista_final=[]
  for lista in puntos:
    lista =  ast.literal_eval(lista)
    lista = [tuple(l) for l in lista]
    lista1 = list(transformer.itransform(lista))
    lista1 = [t[::-1] for t in lista1]
    lista_final.append(lista1)
  coords = np.array(lista_final)
  data['coordinates'] = coords
  return data

In [0]:
"""
Se le pasa un par de coordenadas y las devuelve transformadas
en latitud y longitud
"""
def coordsTransformation(tupla):
  ## La tupla debe ser de este formato (y,x)
  inProj  = Proj("EPSG:25830", preserve_units=True)
  outProj = Proj("EPSG:4326", preserve_units=True)
  # Example coordinates [729643.21, 4370568.147]
  y,x = tupla
  return(transform(inProj,outProj,y,x))
coordsTransformation((730064.725999999791384,4372372.749))

(39.47020700796335, -0.3255057140748624)

In [0]:
"""
Se le pasa un par de coordenadas y devuelve la distancia 
entre la UPV y este par de coordenadas
"""
def distanceOriginal(lista):
  dists = []
  for tupla in lista:
    lat,lon = tupla
    upvLat,upvLon = (39.482921, -0.348383)
    dist = math.sqrt( (upvLat - lat)**2 + (upvLon - lon)**2 )
    dists.append(dist)
  return min(dists)


In [0]:
"""
Lee las coordenadas y crea una columna con la distancia mínima entre cada par de coordenadas
de cada tramo y de la UPV
"""
def distance2upv(data):
  puntos = data.coordinates
  #coords = np.array(list(transformer.itransform(puntos)))
  lista_final=[]
  for lista in puntos:
    lista_final.append(list(map(distanceOriginal,[lista])))

  data['distance2upv'] = np.array(lista_final)
  return data

In [0]:
"""
Esta función lee los archivos json de tramos en la carpeta /tramos, los pasa a 
.csv y los formatea para que tengan un formato adecuado, tiempo en formato español,
elimina campos que no nos interesan, etc...
"""
def tramosToCsv():
  ### Lee los archivos json de la carpeta espiras situada en Drive y los mete todos
  ### en un archivo .csv
  dir = "./tramos/"
  tramos_totales = []
  fileNames = os.listdir(dir)
  #Para cada archivo json lo abrimos, lo procesamos y lo añadimos a un csv
  for fileJson in fileNames:
    with open (dir+fileJson,mode='r',encoding='UTF-8') as f:
      data = json.load(f)
      espiras = data['resources']
    ## Eliminamos el campo 'uri' que no nos interesa
    for espira in espiras:          #Tramos es una lista de diccionarios
      del espira['uri']
    tramos_totales.extend(espiras) # Agregamos a los tramos totales el contenido de los diccionarios
    #print(len(tramos_totales))                                   

  csvAllDicts = 'tramos.csv'
  keys = tramos_totales[0].keys()

  with open (csvAllDicts,mode='w',encoding='UTF-8') as f:
      dict_writer = csv.DictWriter(f, keys)
      dict_writer.writeheader()
      dict_writer.writerows(tramos_totales)
  data = pd.read_csv("tramos.csv")
  data['modified'] = data['modified'].map(timeFormated)     #Cambiamos la fecha de datos a la fecha formateada
  data = coordsColumnsTransformation(data)                  #Transformamos las coordenadas a formato latitud, longitud
  data = distance2upv(data)
  data.to_csv(csvAllDicts, sep=',', encoding='utf-8',index=False)

tramosToCsv()

In [0]:
# Mostramos los tramos más cercanos a el sensor de la UPV
data = pd.read_csv('tramos.csv')
data.nsmallest(10, ['distance2upv']) 

# Una vez dibujados todos los sensores en el maps
# el que nos interesa es acceso barcelona ramal salida 
# de paso inferiorm id = a297

,des_tramo,idtramo,modified,lectura,coordinates,distance2upv
81,ACCESO BARCELONA RAMAL DE ENTRADA [De V-21 a R...,a296,2020-03-18 12:30:04,240,"[(39.48282867675427, -0.34921370912423266), (3...",0.000836
256,ACCESO BARCELONA RAMAL DE SALIDA [De Paso Infe...,a297,2020-03-18 12:30:04,30,"[(39.48282845014904, -0.34921404324625727), (3...",0.000836
335,ACCESO BARCELONA (PASO INFERIOR) [Entre V-21 y...,a287,2020-03-18 12:30:04,684,"[(39.48282845014904, -0.34921404324625727), (3...",0.000836
367,ACCESO BARCELONA Entre V-21 y Ronda Norte,a1,2020-03-18 12:30:04,216,"[(39.48282845014904, -0.34921404324625727), (3...",0.000836
187,"NARANJOS, AVENIDA DE LOS",a165,2020-03-18 12:30:04,585,"[(39.47987052051646, -0.34528370155352395), (3...",0.002482
257,AV. CATALUÑA [De Rotonda a Primado Reig],a3,2020-03-18 12:30:04,108,"[(39.4822649217558, -0.3513195457935688), (39....",0.003009
322,RONDA NORTE,a376,2020-03-18 12:30:04,921,"[(39.494270843998216, -0.3576014919009384), (3...",0.003009
90,AVENIDA LOS NARANJOS,a360,2020-03-18 12:30:04,594,"[(39.47987052051646, -0.34528370155352395), (3...",0.004349
162,RAMON LLULL,a212,2020-03-18 12:30:04,199,"[(39.47987052051646, -0.34528370155352395), (3...",0.004349
154,CLARIANO,a295,2020-03-18 12:30:04,288,"[(39.47856063403376, -0.3511765673562111), (39...",0.004645


In [0]:
"""Esta función guarda las coordenadas del tramo indicado en un csv
  que posteriormente es importado a Google Maps y es mostrado ahí
"""
def coordSensor(sensor):
  row =data.loc[data['idtramo'] ==sensor]
  coordenadas = row['coordinates'].tolist()

  coordenadas =  ast.literal_eval(coordenadas[0])
  filepath = './coordenadasTramos/'
  with open (filepath +sensor+'.csv',mode='w',encoding='UTF-8') as file:
        file_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for i in coordenadas:
            file_writer.writerow(i)

coordSensor("a3")

In [0]:
"""Esta función guarda las coordenadas del tramo indicado en un csv
  que posteriormente es importado a Google Maps y es mostrado ahí
"""
def coordSensor():
  coordenadas=[]
  for listaOfTuplas in data["coordinates"]:
    coordenadas.extend(ast.literal_eval(listaOfTuplas)) 

  filepath = './coordenadasTramos/'
  with open (filepath +"coordendasTramos"+'.csv',mode='w',encoding='UTF-8') as file:
        file_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for i in coordenadas:
            file_writer.writerow(i)



coordSensor()

ERROR! Session/line number was not unique in database. History logging moved to new session 60
